In [22]:
%pylab inline


Populating the interactive namespace from numpy and matplotlib


First from Hoffman's book:

1. Assume a trial value $\mathbf{x}^{(0)}$. Choose one component to be unity. Designate that the unity component.

2. Perform the matrix multiplication: $\mathbf{A}\mathbf{x}^{(0)} = \mathbf{y}^{(1)}$

3. Scale $\mathbf{y}^{(1)}$ so that the unity component remains unity:

$$\mathbf{y}^{(1)} = \lambda^{(1)} \mathbf{x}^{(1)}$$

4. Repeat: 

$$\mathbf{A}\mathbf{x}^{(k)} = \mathbf{y}^{(k+1)} = \lambda^{(k+1)} \mathbf{x}^{(k+1)}$$


In [23]:
def hoffman_power(A):
    n = A.shape[0]
    x = ones(n)
    lmda = 0
    for i in range(30):
        last = lmda
        y = dot(A,x)
        lmda = y[0]
        x = y/lmda
    error = last - lmda    
    return (lmda, x, abs(error))    
    

A = array([[8,-2,-2],[-2,4,-2],[-2,-2,13]])
hoffman_power(A)

(13.870603889183611,
 array([ 1.        ,  0.49178084, -3.42707923]),
 1.1429145457597656e-05)

In [24]:
eig(A)[0]

array([  2.5089808 ,   8.62043408,  13.87058512])

###Inverse Power Method

1. Solve for $\mathbf{L}$ and $\mathbf U$ such that $\mathbf{LU} = \mathbf{A}$

2. Assume $\mathbf{x}^{(0)}$. Designate a component of $\mathbf{x}$ to be unity.

3. Solve for $\mathbf{x}'$ by forward substitution: $\mathbf{L} \mathbf{x}' = \mathbf{x}^{(0)}$

4. Solve for $\mathbf{y}^{(1)}$ by back substitution: $\mathbf{U}\mathbf{y}^{(1)} =  \mathbf{x}' $

5. Scale $\mathbf{y}^{(1)}$ so that the unity component is unity. Thus, $\mathbf{y}^{(1)} = \lambda_{\text{inverse}}^{(1)} \mathbf{x}^{(1)} $

6. Repeat

$$\mathbf{L} \mathbf{x}' = \mathbf{x}^{(k)}$$

$$ \mathbf{U}\mathbf{y}^{(k+1)} =  \mathbf{x}'$$

$$ \mathbf{y}^{(k+1)} = \lambda_{\text{inverse}}^{(k+1)} \mathbf{x}^{(k+1)} $$

In [25]:
from scipy.linalg import lu


def hoffman_inverse_power(A):
    l,u = lu(A, permute_l=True)
    n = A.shape[0]
    x = ones(n)
    x_prime = ones(n)
    y = ones(n)
    lmda = 0
    
    for i in range(10):
        last = lmda
        for j in range(n):
            x_prime[j] = x[j]/l[j][j]
            for k in range(j):
                x_prime[j] -= x_prime[k]*l[j][k]

        for j in reversed(range(n)):
            y[j] = x_prime[j]/u[j][j]
            for k in range(j+1, n):
                y[j] -= y[k]*u[j][k]/u[j][j]

        lmda = y[0] 
        x = y/lmda

    return (1/lmda, x, abs(lmda-last))    
    
A = array([[8,-2,-2],[-2,4,-2],[-2,-2,13]])
hoffman_inverse_power(A)    

(2.5090028139770366,
 array([ 1.        ,  2.14578754,  0.59971106]),
 8.4845877590389307e-06)

###Shifted Power Method

The eigenvalues of a matrix $\mathbf{A}$ may be shifted by a scalar s by subtracting $s\mathbb{I} \mathbf{x} = s \mathbf{x}$ from both sides of the standard eigenproblem, $\mathbf{A} \mathbf{x} = \lambda \mathbf{x} $. Thus,

$$\mathbf{A} \mathbf{x}-s\mathbb{I}\mathbf{x}  = \lambda \mathbf{x} -s \mathbf{x}$$

$$(\mathbf{A} - s\mathbb{I})\mathbf{x}  = (\lambda -s )\mathbf{x}$$

$$\mathbf{A}_{\text{shifted}} \mathbf{x} = \lambda_{\text{shifted}} \mathbf{x}$$

Shifting the eigenvalues of a matrix can be used to:

1. Find the opposite extreme eigenvalue.

2. Find intermediate eigenvalues

3. Accelerate convergence 

In [26]:
A = array([[8,-2,-2],[-2,4,-2],[-2,-2,13]])
value, vector, error = hoffman_power(A)
A_shifted = A - eye(3)*value
value2, vector, error = hoffman_power(A_shifted)
value2 +value

2.508980800011047

##Direct Method

The characteristic equation is obtained from

$$\text{det}(\mathbf{A} - \lambda \mathbb{I}) = 0$$

Rather than solving for the roots of the characteristic equation directly, we instead solve the characteristic equation iteratively (my sentences). "This can be accomplished my applying the secant method. Two intitial approximations of $\lambda$ are assumed, $\lambda_0$ and $\lambda_1$, the corresponding values of the cahracteristic determinant are computed, and these results are used to construct a linear relationship between $\lambda$ and the value of the characteristic determinant. The solution of that linear relationship is taken as the next approximation to $\lambda$, and the procedure is repeated to convergence

In [27]:
def direct_method(A, guess_1, guess_2):
    if(abs(guess_2 - guess_1)< 1e-5):
        return guess_2
    else:
        n = A.shape[0]
        f_1 = det(A - eye(n)*guess_1)
        f_2 = det(A - eye(n)*guess_2)
        slope = (f_2 - f_1)/(guess_2 - guess_1)
        guess_3 = guess_2 - f_2/slope
        return direct_method(A, guess_2, guess_3)
    
    
direct_method(A,15, 13)

13.870585123309459

##QR Method


$$A = \left[ \begin{matrix} a_{11} ~ a_{12}~ ... ~a_{1n}\\ a_{21} ~a_{22} ~... ~a_{2n}\\ . ~~ . ~~ .\\ a_{n1}  ~ a_{n2} ~ ... ~ a_{nn} \end{matrix} \right]= [\mathbf{a}_1 ~~ \mathbf{a}_2 ~ . ~ . ~ .  ~~  \mathbf{a}_n]$$


Use the Gram-Schmidt process:

$$\mathbf{q}_1= \frac{\mathbf{a}_1} {||\mathbf{a}_1||}$$

$$\mathbf{a}'_2 = \mathbf{a}_2 - (\mathbf{q}_1^T \mathbf{a}_2) \mathbf{q}_1$$

$$\mathbf{q}_2= \frac{\mathbf{a}'_2} {||\mathbf{a}'_2||}$$

$$\mathbf{a}'_3 = \mathbf{a}_3 - (\mathbf{q}_1^T \mathbf{a}_3) \mathbf{q}_1 - (\mathbf{q}_2^T \mathbf{a}_3) \mathbf{q}_2$$

$$\mathbf{q}_3= \frac{\mathbf{a}'_3} {||\mathbf{a}'_3||}$$

and so on...

The matrix <b>Q</b> is composed of the column vectors

The matrix <b>R</b> is assembled from the elements computed in the evaluation of <b>Q</b>

$$r_{i,i} = ||\mathbf{a}'_i||$$

$$r_{i,j} = \mathbf{q}_i^T \mathbf{a}_j ~~~~~~ \text{where} ~~~~ j=i+1, ..., n$$

The first step in the QR process is to set $\mathbf{A}^{(0)} = \mathbf{A}$ and factor $\mathbf{A}^{(0)}$ by the Gram-Schmidt process into $\mathbf{Q}^{(0)}$ and $\mathbf{R}^{(0)}$. The next step is to reverse the factors  $\mathbf{Q}^{(0)}$ and $\mathbf{R}^{(0)}$ to obtain

$$\mathbf{A}^{(1)} =  \mathbf{R}^{(0)} \mathbf{Q}^{(0)} $$

$\mathbf{A}^{(1)}$ is similar to $\mathbf{A}$, so the eigenvalues are preserved. $\mathbf{A}^{(1)}$ is factored by the Gram-Schmidt process to obtain $\mathbf{Q}^{(1)}$ and $\mathbf{R}^{(1)}$, and the factors are reversed to obtain $\mathbf{A}^{(2)}$. Thus,

$$\mathbf{A}^{(2)} =  \mathbf{R}^{(1)} \mathbf{Q}^{(1)} $$

The process is continued to determine $\mathbf{A}^{(3)}, \mathbf{A}^{(4)}, ... , \mathbf{A}^{(n)}$. When $\mathbf{A}^{(n)}$ approaches triangular form, within some tolerance, the eigenvalues of $\mathbf{A}$ are the diagonal elements.

In [52]:
def hoffman_QR(a):
    A = a.astype(float)
    n = A.shape[0]
    q = zeros((n,n))
    r = zeros((n,n))
    a_prime = zeros(n)
    for k in range(20):
        for i in range(n):
            a_prime = A[:,i]
            for j in range(i):
                r[j][i] = dot(q[:,j], A[:,i])
                a_prime -= r[j][i]*q[:,j]
            r[i][i] = norm(a_prime)    
            q[:,i] = a_prime/r[i][i]  
        A = dot(r,q)
    return array([A[i][i] for i in range(n)])
    
    
A = array([[8,-2,-2],[-2,4,-2],[-2,-2,13]])    
hoffman_QR(A)    

array([ 13.87058484,   8.62043436,   2.5089808 ])